In [1]:
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_semantics.zip
!unzip data_semantics.zip -d data_semantics

--2021-07-19 10:58:28--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_semantics.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.169.25
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.169.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 327699796 (313M) [application/zip]
Saving to: ‘data_semantics.zip’

data_semantics.zip  100%[===================>] 312.52M  19.0MB/s    in 18s     

2021-07-19 10:58:47 (17.5 MB/s) - ‘data_semantics.zip’ saved [327699796/327699796]

Archive:  data_semantics.zip
   creating: data_semantics/testing/
   creating: data_semantics/testing/image_2/
  inflating: data_semantics/testing/image_2/000000_10.png  
  inflating: data_semantics/testing/image_2/000001_10.png  
  inflating: data_semantics/testing/image_2/000002_10.png  
  inflating: data_semantics/testing/image_2/000003_10.png  
  inflating: data_semantics/testing/image_2/000004_10.png  
  inflating

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as pyplt
import matplotlib.image as mpimg
import cv2
from glob import glob
from albumentations import HorizontalFlip, VerticalFlip, OpticalDistortion, ElasticTransform,GridDistortion

def pathloader(address):
    train_x = sorted(glob(os.path.join(address, 'training', 'image_2', '*.png')))
    train_y = sorted(glob(os.path.join(address, 'training', 'semantic_rgb', '*.png')))

    return (train_x, train_y)


def create_dir(address):
    if not os.path.exists(address):
        os.makedirs(address)


def augment(image, mask, savepath,augment = True):
    H = 512
    W = 512

    for (x, y) in zip(image, mask):
      name = x.split('/')[-1].split('.')[0]

      x = cv2.imread(x, cv2.IMREAD_COLOR)
      y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
      aug = HorizontalFlip(p=1.0)
      augmented = aug(image=x, mask=y)
      x1 = augmented['image']
      y1 = augmented['mask']

      aug = VerticalFlip(p=1.0)
      augmented = aug(image=x, mask=y)
      x2 = augmented['image']
      y2 = augmented['mask'] 

      aug = ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03)
      augmented = aug(image=x, mask=y)
      x3 = augmented['image']
      y3 = augmented['mask']

      aug = GridDistortion(p=1)
      augmented = aug(image=x, mask=y)
      x4 = augmented['image']
      y4 = augmented['mask']

      aug = OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)
      augmented = aug(image=x, mask=y)
      x5 = augmented['image']
      y5 = augmented['mask']

      X = [x, x1, x2, x3, x4, x5]
      Y = [y, y1, y2, y3, y4, y5]

      index = 1

      for i, j in zip(X, Y):
         i = cv2.resize(i, (W, H))
         j = cv2.resize(j, (W, H))

         image_name = f"{name}_{index}.png"
         image_mask_name = f"{name}_{index}.png"

         image_name_location = os.path.join(savepath, 'new_train', image_name)
         image_mask_name_location = os.path.join(savepath, 'new_train_mask', image_mask_name)

         cv2.imwrite(image_name_location, i)
         cv2.imwrite(image_mask_name_location, j)

         index += 1



create_dir(os.path.join('data_semantics',"new_train"))
create_dir(os.path.join('data_semantics',"new_train_mask"))
X,Y = pathloader('data_semantics')
print("going to augment")
augment(X,Y,'data_semantics')
print("done")








going to augment
done


In [8]:
valid_path = os.path.join("data_semantics","valid")
valid_path_mask = os.path.join("data_semantics","valid_mask")
create_dir(valid_path)
create_dir(valid_path_mask)



In [ ]:
def iou(y_true,y_prev):
    def f(y_true,y_pred):
        intersection = (y_true*y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = intersection/union
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f,[y_true,y_prev],tf.float32)

def dice_coef(y_true,y_pred):
    y_true = keras.layers.Flatten()(y_true)
    y_pred = keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true*y_pred)
    dice_coeff = (2*intersection)/(tf.reduce_sum(y_true)+tf.reduce_sum(y_pred))
    return  dice_coeff

def dice_loss(y_true,y_pred):
    return 1.0-dice_coef(y_true,y_pred)

In [ ]:
def down_block(x,filters,filter_size = (3,3),stride = 1,padding = 'same'):

  c = keras.layers.Conv2D(filters,filter_size,strides=stride,padding=padding,activation='relu')(x)
  c = keras.layers.Conv2D(filters,filter_size,strides=stride,padding=padding,activation='relu')(c)
  p = keras.layers.MaxPool2D((2,2),(2,2))(c)
  return c,p

def up_block(x,skip_layer,filters,filter_size = (3,3),stride = 1,padding = 'same'):

  up_sample = keras.layers.UpSampling2D((2,2))(x)
  concatnt = keras.layers.Concatenate()([up_sample,skip_layer])
  c = keras.layers.Conv2D(filters,filter_size,strides=stride,padding=padding,activation='relu')(concatnt)
  c = keras.layers.Conv2D(filters,filter_size,strides=stride,padding=padding,activation='relu')(c)
  return c


def bottleneck(x,filters,filter_size = (3,3),stride = 1,padding = 'same'):
  c = keras.layers.Conv2D(filters,filter_size,strides=stride,padding=padding,activation='relu')(x)
  c = keras.layers.Conv2D(filters,filter_size,strides=stride,padding=padding,activation='relu')(c)
  return c

def UNet():
  inputs = keras.layers.Input((512,512,3))

  p0 = inputs
  print(p0.shape)
  c1,p1 = down_block(p0,64)
  print(p1.shape)
  c2,p2 = down_block(p1,128)
  print(p2.shape)
  c3,p3 = down_block(p2,256)#16
  print(p3.shape)
  c4,p4 = down_block(p3,512)#8
  print(p4.shape)

  c6 = bottleneck(p4,1024)

  c7 = up_block(c6,c4,512)
  print(c7.shape)
  c8 = up_block(c7,c3,256)
  print(c8.shape)
  c9 = up_block(c8,c2,128)
  print(c9.shape)
  c10 = up_block(c9,c1,64)
  print(c10.shape)

  outputs = keras.layers.Conv2D(1,(1,1),(1,1),padding='same',activation = 'sigmoid')(c10)
  print(outputs.shape)

  model = keras.models.Model(inputs,outputs,name="UNet")
  return model

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision



(H,W) = (512,512)
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def pathloader(address):
    x = sorted(glob(os.path.join(address, 'training','image_2','*.png')))
    y = sorted(glob(os.path.join(address, 'new_train_mask', '*.png')))
    return x, y
def pathloadervalid(address):
    x = sorted(glob(os.path.join(address, 'valid','*.png')))
    y = sorted(glob(os.path.join(address, 'valid_mask', '*.png')))
    return x, y

def shuffling(x,y):
    x,y = shuffle(x,y,random_state=31)
    return x,y

def read_image(path):
    path = path.decode()
    image = cv2.imread(path,cv2.IMREAD_COLOR)
    image = image/255.0
    image = image.astype(np.float32)
    return image

def read_mask(path):
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    for i in range(len(image[:,0])):
        for j in range(len(image[0,:])):
            if(image[i,j]==26):
                image[i,j] = 1
            else:
                image[i,j]=0
    image = image.astype(np.float32)
    image = np.expand_dims(image,axis = -1)
    return image

def tf_read(x,y):
    def f(x,y):
        x = read_image(x)
        y = read_mask(y)
        return x,y
    x,y = tf.numpy_function(f,[x,y],[tf.float32,tf.float32])
    x.set_shape([H,W,3])
    y.set_shape([H, W, 1])
    return x,y

def tf_dataset(X,Y,batchsize):
    dataset = tf.data.Dataset.from_tensor_slices((X,Y))
    dataset = dataset.map(tf_read)
    dataset = dataset.batch(batchsize)
    dataset = dataset.prefetch(5)
    return dataset


np.random.seed(31)
tf.random.set_seed(31)

create_dir("files")

batch_size = 2
learning_rate = 0.0005
epochs = 5
model_path = os.path.join("files","model.h5") #for weights
csv_path = os.path.join("files","data.csv")

dataset_path = "data_semantics"
train_x,train_y = pathloader(dataset_path)
train_x,train_y = shuffling(train_x,train_y)
valid_x,valid_y = pathloadervalid(dataset_path)

dataset = tf_dataset(train_x,train_y,batch_size)
valid_dataset = tf_dataset(valid_x,train)

train_steps = len(train_x)//batch_size
valid_steps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_steps += 1

model = UNet()
model.compile(loss = "binary_crossentropy",optimizer = Adam(learning_rate),metrics = [dice_coef,iou,Recall(),Precision()])

callbacks = [
    ModelCheckpoint(model_path,verbose=1,save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss",patience = 5,verbose = 1,factor = 0.1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_loss",patience=10)
]
model.fit(   dataset,   epochs = epochs,    steps_per_epoch = train_steps,callbacks = callbacks)

(None, 512, 512, 3)
(None, 256, 256, 64)
(None, 128, 128, 128)
(None, 64, 64, 256)
(None, 32, 32, 512)
(None, 64, 64, 512)
(None, 128, 128, 256)
(None, 256, 256, 128)
(None, 512, 512, 64)
(None, 512, 512, 1)
Epoch 1/5
3/3 [==============================] - 164s 53s/step - loss: 0.5199 - dice_coef: 8.6169e-05 - iou: 4.3086e-05 - recall_1: 0.0267 - precision_1: 8.2874e-05
Epoch 2/5
3/3 [==============================] - 162s 53s/step - loss: 0.0698 - dice_coef: 0.0000e+00 - iou: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00
Epoch 3/5
3/3 [==============================] - 164s 53s/step - loss: 0.0310 - dice_coef: 2.5645e-22 - iou: 1.2822e-22 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00
Epoch 4/5
3/3 [==============================] - 161s 52s/step - loss: 0.0120 - dice_coef: 1.4703e-05 - iou: 7.3514e-06 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00
Epoch 5/5
3/3 [==============================] - 161s 53s/step - loss: 0.0050 - dice_coef: 2.1797e-05 - iou: 1.0899e-05 